## GPU Check

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct 30 11:16:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0  On |                  N/A |
| 26%   56C    P0    48W / 175W |   3159MiB /  8192MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Gradio

In [2]:
! pip install gradio

## Aggregate, process, clean, label, and version data

In [3]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def is_cat(x): return x[0].isupper() 
dls = ImageDataLoaders.from_name_func(
    path,
    files,
    pat='(.+)_\d+.jpg',
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    label_func=is_cat, 
    item_tfms=Resize(192),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

## Find model arch and weights / Write and debug model code

In [4]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(1)
learn.path = Path('.')
learn.export('model.pkl')

/home/thomas/miniconda3/envs/dogcat/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
/home/thomas/miniconda3/envs/dogcat/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,error_rate,time
0,0.205011,0.059007,0.022327,00:14


epoch,train_loss,valid_loss,error_rate,time
0,0.075619,0.028657,0.012179,00:18


## Deploy model

In [5]:
learn = load_learner('model.pkl')

In [6]:
categories = ('Dog', 'Cat')

In [7]:
def classify_image(img):
    pred, idx, probs = learn.predict(img)
    return dict(zip(categories, map(float, probs)))

In [8]:
#hide_output
import gradio as gr

image = gr.inputs.Image(shape=(192, 192))
label = gr.outputs.Label()
examples = ['Dog.jpg', 'cat.jpg', 'dunno.jpg']
title = "Dogs V Cats Classifier"
description = "A classifier trained on the Oxford Pets dataset with fastai. Created as a demo for Gradio and HuggingFace Spaces."
interpretation='default'
enable_queue=True

intf = gr.Interface(
    fn=classify_image,
    inputs=image,
    outputs=label,
    examples=examples,
    title=title,
    description=description,
    interpretation=interpretation,
    enable_queue=enable_queue
)

intf.launch(inline=True, share=True)

/home/thomas/.local/lib/python3.8/site-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/thomas/.local/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/thomas/.local/lib/python3.8/site-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/home/thomas/.local/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/home/thomas/.local/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warni

Running on local URL:  http://127.0.0.1:7861/
Running on public URL: https://afaa023163ce7d1f.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://www.huggingface.co/spaces)


(<gradio.routes.App at 0x7f3f6d85f730>,
 'http://127.0.0.1:7861/',
 'https://afaa023163ce7d1f.gradio.app')